# Featurize a dataset

Any machine learning model will expect tensorial representations of the chemical data. This notebooks provides a workflow to achieve such goal.

`kinoml.dataset.DatasetProvider` objects need to be available to deal with your collection of raw measurements for protein:ligand systems. These objects are, roughly, a list of `kinoml.core.BaseMeasurement`, each containing a set of `.values` and a some extra metadata, like the `system` objects to be featurized here.

In ligand-based models, protein information is only considered marginally, and most of the action happens at the ligand level. Usually starting with a string representation such as SMILES, or a database identifier such as a PubChem ID, these are promoted to (usually) RDKit objects and then transformed into a tensor of some form (e.g. fingerprints, molecular graph as an adjacency matrix, etc).

Available featurizers can be found under `kinoml.features`.

## How to use

Run `python run_notebook.py --help` for more information.

In [1]:
# If this is the template file (and not a copy) and you are introducing changes,
# update VERSION with the current date (YYYY.MM.DD)
VERSION = "2021.04.21" 

## ✏ Define hyper parameters

In [2]:
# TEMPLATE VALUES -- these are overriden (see below if executed) by papermill using a YAML or Python file as input
DATASET_CLS = "import.path.to.DatasetProvider"
DATASET_KWARGS = {"option": "value", "option2": "value2"}

PIPELINES = {
    "someuniquekey": [
        ("import.path.to.SomeFeaturizer", {"option": "value", "option2": "value2"}),
        ("import.path.to.SomeOtherFeaturizer", {"option": "value", "option2": "value2"}),
    ]
}
PIPELINES_AGG = "kinoml.features.core.Concatenated"
PIPELINES_AGG_KWARGS = {}

GROUPS = [
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda something: something.attribute"}),
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda otherthing: otherthing.attribute2"})
]

TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}

## IGNORE THIS ONE
HERE = _dh[-1]

In [3]:
# Parameters
DATASET_CLS = "kinoml.datasets.chembl.ChEMBLDatasetProvider"
DATASET_KWARGS = {
    "path_or_url": "https://github.com/openkinome/kinodata/releases/download/v0.2/activities-chembl28-sample100_v0.2.zip"
}
PIPELINES = {
    "ligand": [
        ["kinoml.features.ligand.SmilesToLigandFeaturizer", {}],
        ["kinoml.features.ligand.OneHotSMILESFeaturizer", {}],
        ["kinoml.features.core.PadFeaturizer", {"shape": "auto"}],
    ]
}
PIPELINES_AGG = "kinoml.features.core.Concatenated"
PIPELINES_AGG_KWARGS = {}
FEATURIZE_KWARGS = {}
GROUPS = [
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: measurement.system.protein.name"},
    ],
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: type(measurement).__name__"},
    ],
]
TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}
HERE = "/home/jaime/devel/py/openkinome/experiments-binding-affinity/features/example-ligand-only-chembl28-ohesmiles-1k-subsample"


⚠ From here on, you should _not_ need to modify anything else 🤞

---

Define key paths for data and outputs:

In [4]:
from pathlib import Path

HERE = Path(HERE)
for parent in HERE.parents:
    if next(parent.glob(".github/"), None):
        REPO = parent
        break

# Generate paths for this pipeline
featurizer_path = []
for name, branch in PIPELINES.items():
    featurizer_path.append(name)
    for clsname, kwargs in branch:
        clsname = clsname.rsplit(".", 1)[1]
        kwargs = [f"{k}={''.join(c for c in str(v) if c.isalnum())}" for k,v in kwargs.items()]
        featurizer_path.append("_".join([clsname] + kwargs))

OUT = HERE / "_output"  / "__".join(featurizer_path) / DATASET_CLS.rsplit('.', 1)[1]
OUT.mkdir(parents=True, exist_ok=True)

print(f"This notebook:           HERE = {HERE}")
print(f"This repo:               REPO = {REPO}")
print(f"Outputs in:               OUT = {OUT}")

This notebook:           HERE = /home/jaime/devel/py/openkinome/experiments-binding-affinity/features/example-ligand-only-chembl28-ohesmiles-1k-subsample
This repo:               REPO = /home/jaime/devel/py/openkinome/experiments-binding-affinity
Outputs in:               OUT = /home/jaime/devel/py/openkinome/experiments-binding-affinity/features/example-ligand-only-chembl28-ohesmiles-1k-subsample/_output/ligand__SmilesToLigandFeaturizer__OneHotSMILESFeaturizer__PadFeaturizer_shape=auto/ChEMBLDatasetProvider


In [5]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparametrs) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

## Setup is finished, start working

In [6]:
from warnings import warn
import os
import sys
from pathlib import Path
from datetime import datetime

import numpy as np

from kinoml.utils import seed_everything, import_object
seed_everything();
print("Run started at", datetime.now())

Run started at 2021-04-21 10:39:39.073911


## Load raw data

> This `import_object` function allows us to take a `str` containing a Python import path (e.g. `kinoml.datasets.chembl.ChEMBLDatasetProvider`) and obtain the imported object directly. That's how we can encode classes in JSON-only `papermill` inputs.
>
> See the help message `import_object?` for more info.

In [7]:
dataset = import_object(DATASET_CLS).from_source(**DATASET_KWARGS)
dataset

  0%|          | 0/100 [00:00<?, ?it/s]

<ChEMBLDatasetProvider with 100 measurements (pIC50Measurement=33, pKiMeasurement=63, pKdMeasurement=4), and 100 systems (AminoAcidSequence=15, SmilesLigand=97)>

In [8]:
df = dataset.to_dataframe()
df

,Systems,n_components,Measurement,MeasurementType
0,O75116 & CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4...,2,14.096910,pIC50Measurement
1,O75116 & CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4...,2,14.000000,pIC50Measurement
2,O75116 & CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4...,2,14.000000,pIC50Measurement
3,O75116 & CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4...,2,13.958607,pIC50Measurement
4,O75116 & CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4...,2,13.920819,pIC50Measurement
...,...,...,...,...
95,P48736 & COc1cncc(N2CCc3nc(NC(=O)NCCc4cn(CCF)c...,2,11.221849,pKiMeasurement
96,Q16539 & CC(C)(C)c1cc(NC(=O)NCc2ccccc2Sc2ccc3n...,2,12.000000,pKdMeasurement
97,P31749 & Cc1ccc(-c2nc(SCC(=O)O)c3c(=O)n(C)c(=O...,2,11.892790,pKdMeasurement
98,P68400 & CC(=O)Nc1cc(Nc2cc(NC3CC3)n3ncc(C#N)c3...,2,11.308919,pKdMeasurement


## Featurize

In [9]:
# build pipeline
from kinoml.features.core import Pipeline

pipelines = []
for key, pipeline_instructions in PIPELINES.items():
    print(f"Building featurizer `{key}` with instructions:")
    featurizers = []
    for featurizer_import_str, kwargs in pipeline_instructions:
        kwargs = kwargs or {}  # make sure empty values (None, "") turn into {} so we can do **kwargs below
        print(f"  Instantiating `{featurizer_import_str}` with options `{kwargs}`")
        featurizers.append(import_object(featurizer_import_str)(**kwargs))
    pipelines.append(Pipeline(featurizers))
print("Resulting pipelines:", *pipelines)
aggregated_pipeline = import_object(PIPELINES_AGG)(pipelines, **PIPELINES_AGG_KWARGS)
print("Aggregated pipelines:", aggregated_pipeline)

Building featurizer `ligand` with instructions:
  Instantiating `kinoml.features.ligand.SmilesToLigandFeaturizer` with options `{}`
  Instantiating `kinoml.features.ligand.OneHotSMILESFeaturizer` with options `{}`
  Instantiating `kinoml.features.core.PadFeaturizer` with options `{'shape': 'auto'}`
Resulting pipelines: <Pipeline([SmilesToLigandFeaturizer, OneHotSMILESFeaturizer, PadFeaturizer])>
Aggregated pipelines: <Concatenated([Pipeline([SmilesToLigandFeaturizer, OneHotSMILESFeaturizer, PadFeaturizer])])>


In [10]:
# prefeaturize everything
aggregated_pipeline.featurize(dataset.systems);

## Filter

Remove systems that couldn't be featurized. Successful featurizations are stored in `measurement.system.featurizations['last']` so we test for that key existence.

In [11]:
from kinoml.datasets.groups import CallableGrouper, RandomGrouper
grouper = CallableGrouper(lambda measurement: 'invalid' if 'last' not in measurement.system.featurizations else 'valid')
grouper.assign(dataset, overwrite=True, progress=False)
groups = dataset.split_by_groups()
if "invalid" in groups:
    _invalid = groups.pop("invalid")
    warn(f"{len(_invalid)} entries could not be featurized!. Possible errors:")
    warn(f"{_invalid[0].system.featurizations}")

## Groups

Cumulatively apply groups.

In [12]:
groups[("valid",)] = groups.pop("valid")
for grouper_str, grouper_kwargs in GROUPS:
    grouper_cls = import_object(grouper_str)
    ## We need this because lambda functions are not JSON-serializable
    if issubclass(grouper_cls, CallableGrouper):
        for k, v in list(grouper_kwargs.items()):
            if k == "function" and isinstance(v, str):
                grouper_kwargs[k] = eval(v)  # sorry :)
    ## End of lambda hack
    grouper = grouper_cls(**grouper_kwargs)        
    for group_key in list(groups.keys()):
        grouper.assign(groups[group_key], overwrite=True, progress=False)
        for subkey, subgroup in groups.pop(group_key).split_by_groups().items():
            groups[group_key + (subkey,)] = subgroup
print("10 groups to show keys:", *list(groups.keys())[:10], sep="\n")

10 groups to show keys:
('valid', 'O75116', 'pIC50Measurement')
('valid', 'O75116', 'pKdMeasurement')
('valid', 'P23458', 'pIC50Measurement')
('valid', 'Q96Q04', 'pIC50Measurement')
('valid', 'P00533', 'pIC50Measurement')
('valid', 'O60674', 'pIC50Measurement')
('valid', 'O60674', 'pKiMeasurement')
('valid', 'P28482', 'pIC50Measurement')
('valid', 'P50750', 'pIC50Measurement')
('valid', 'P52333', 'pIC50Measurement')


## Write tensors to disk

Output files are written to `_output/<PIPELINE>/<DATASET>/<GROUP>.npz` files.

Each `npz` will contain two `np.ndarray` objects: `X` (featurized systems) and `y` (associated measurements), plus the train/test/validation indices.

In [13]:
random_grouper = RandomGrouper(TRAIN_TEST_VAL_KWARGS)

for group, ds in sorted(groups.items(), key=lambda kv: len(kv[1]), reverse=True):
    indices = random_grouper.indices(ds)
    X = np.asarray(ds.featurized_systems())
    y = ds.measurements_as_array()
    np.savez(OUT / f"{'__'.join([g for g in group if g != 'valid'])}.npz", X=X, y=y.astype("float32"), **indices)

In [14]:
print("Run finished at", datetime.now())

Run finished at 2021-04-21 10:39:40.295765


# Reproducibility logs

In [15]:
from kinoml.utils import watermark
w = watermark()

Watermark
---------
Last updated: 2021-04-21T10:39:40.334074+02:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 4.19.128-microsoft-standard
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Hostname: jrodriguez

Git hash: 8178e91af5b8d96c4f3ece683237a3a256e4e6d0

numpy: 1.20.2
sys  : 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]

Watermark: 2.2.0


nvidia-smi
----------
stdout:
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

conda info
----------


sys.version: 3.7.6 | packaged by conda-forge | (defau...
sys.prefix: /opt/miniconda
sys.executable: /opt/miniconda/bin/python
conda location: /opt/miniconda/lib/python3.7/site-packages/conda
conda-build: /opt/miniconda/bin/conda-build
conda-convert: /opt/miniconda/bin/conda-convert
conda-debug: /opt/miniconda/bin/conda-debug
conda-develop: /opt/miniconda/bin/conda-develop
conda-env: /opt/miniconda/bin/conda-env
conda-index: /opt/miniconda/bin/conda-index
conda-inspect: /opt/miniconda/bin/conda-inspect
conda-metapackage: /opt/miniconda/bin/conda-metapackage
conda-render: /opt/miniconda/bin/conda-render
conda-server: /opt/miniconda/bin/conda-server
conda-skeleton: /opt/miniconda/bin/conda-skeleton
conda-smithy: /opt/miniconda/bin/conda-smithy
user site dirs: ~/.local/lib/python3.8
                ~/.local/lib/python3.7
                ~/.local/lib/python3.6

CIO_TEST: <not set>
CONDA_DEFAULT_ENV: experiments-binding-affinity
CONDA_EXE: /opt/miniconda/bin/conda
CONDA_PREFIX: /home/jaime/.

# packages in environment at /home/jaime/.conda/envs/experiments-binding-affinity:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                      1_llvm    conda-forge
_py-xgboost-mutex         2.0                       cpu_0    conda-forge
absl-py                   0.12.0             pyhd8ed1ab_0    conda-forge
aiohttp                   3.7.4            py38h497a2fe_0    conda-forge
alabaster                 0.7.12                   pypi_0    pypi
amberlite                 16.0                     pypi_0    pypi
ambertools                20.15                    pypi_0    pypi
ansiwrap                  0.8.4                      py_0    conda-forge
anyio                     2.2.0            py38h578d9bd_0    conda-forge
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
argon2-cffi               20.1.0           py38h497a2fe_2    conda

In [16]:
%%capture cap --no-stderr
w = watermark()

In [17]:
import json

with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str, indent=2)